[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sci-ndp/ndp-ep-py/blob/main/docs/source/tutorials/getting_started.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/sci-ndp/ndp-ep-py/main?filepath=docs%2Fsource%2Ftutorials%2Fgetting_started.ipynb)

# Getting Started with NDP EP Python Client

Welcome to the interactive tutorial for the NDP EP Python client library!

## What is NDP EP?

The National Data Platform (NDP) EP allows you to:
- 🏢 Manage organizations and datasets
- 📊 Register data resources (URLs, S3, Kafka topics)
- 🔍 Search and discover datasets
- 📈 Monitor system health

In [ ]:
# Install the ndp-ep library
!pip install ndp-ep

# Import required libraries
import json
from pprint import pprint
from ndp_ep import APIClient

print("✅ Libraries installed successfully!")

## 1. Setting Up the Client

**Authentication Options:**
1. **API Token** (Recommended): Get from https://nationaldataplatform.org/
2. **Username/Password**: Use your NDP credentials
3. **No Auth**: Limited to public endpoints

In [ ]:
# Configuration
API_BASE_URL = "http://155.101.6.191:8003"

# Option 1: Using API token (uncomment to use)
# API_TOKEN = "your-api-token-here"
# client = APIClient(base_url=API_BASE_URL, token=API_TOKEN)

# Option 2: Using username/password (uncomment to use)
# USERNAME = "your-username"
# PASSWORD = "your-password"
# client = APIClient(base_url=API_BASE_URL, username=USERNAME, password=PASSWORD)

# Option 3: Demo mode (limited functionality)
try:
    client = APIClient(base_url=API_BASE_URL)
    print("✅ Client initialized successfully!")
    print(f"🌐 Connected to: {API_BASE_URL}")
except Exception as e:
    print(f"❌ Failed to initialize client: {e}")
    print("💡 Tip: Make sure the API endpoint is accessible")

## 2. Exploring Organizations

Organizations are containers for datasets and resources.

In [ ]:
# List organizations from different servers
servers = ["local", "global", "pre_ckan"]

print("🏢 Listing Organizations...")
for server in servers:
    try:
        orgs = client.list_organizations(server=server)
        print(f"\n📋 Server '{server}': {len(orgs)} organizations")
        
        # Show first few organizations
        for i, org in enumerate(orgs[:3]):
            print(f"   {i+1}. {org}")
        
        if len(orgs) > 3:
            print(f"   ... and {len(orgs) - 3} more")
            
    except Exception as e:
        print(f"   ❌ Error with {server}: {e}")

## 3. Searching for Datasets

Search is one of the most powerful features.

In [ ]:
# Simple search examples
search_terms = [
    ["climate"],
    ["weather", "temperature"],
    ["data"],
    ["sensor"]
]

print("🔍 Performing Searches...")
for terms in search_terms:
    try:
        results = client.search_datasets(terms=terms, server="global")
        print(f"\n🔎 Search for {terms}: {len(results)} results")
        
        # Show first result details
        if results:
            first = results[0]
            print(f"   📄 Example: {first.get('title', 'No title')}")
            print(f"   🏢 Organization: {first.get('organization', {}).get('title', 'Unknown')}")
            print(f"   📊 Resources: {len(first.get('resources', []))}")
            
    except Exception as e:
        print(f"   ❌ Search failed for {terms}: {e}")

print("\n✅ Search exploration completed!")

## 4. Advanced Search

Use filters and advanced queries for more precise results.

In [ ]:
print("🔍 Advanced Search Examples...")

# Advanced search with filters
advanced_searches = [
    {
        "search_term": "climate,weather",
        "server": "global"
    },
    {
        "dataset_name": "sensor", 
        "server": "global"
    },
    {
        "search_term": "data",
        "filter_list": ["format:CSV"],
        "server": "global"
    }
]

for i, search_data in enumerate(advanced_searches, 1):
    try:
        print(f"\n🔎 Advanced Search #{i}:")
        print(f"   Query: {search_data}")
        
        results = client.advanced_search(search_data)
        print(f"   Results: {len(results)} datasets found")
        
        # Show sample results
        for j, result in enumerate(results[:2]):
            title = result.get('title', result.get('name', 'Untitled'))[:50]
            print(f"      {j+1}. {title}...")
            
    except Exception as e:
        print(f"   ❌ Advanced search failed: {e}")

print("\n✅ Advanced search completed!")

## 5. System Information

Check system health and get connection details.

In [ ]:
print("🏥 System Information...")

# Check system status
try:
    status = client.get_system_status()
    print("\n✅ System Status:")
    pprint(status)
except Exception as e:
    print(f"\n⚠️ System status: {e}")

# Get system metrics
try:
    metrics = client.get_system_metrics()
    print("\n📊 System Metrics:")
    pprint(metrics)
except Exception as e:
    print(f"\n⚠️ System metrics: {e}")

# Get Kafka details
try:
    kafka_details = client.get_kafka_details()
    print("\n📡 Kafka Details:")
    pprint(kafka_details)
except Exception as e:
    print(f"\n⚠️ Kafka details: {e}")

print("\n💡 Note: Some endpoints require authentication")

## 6. Creating Resources (Requires Authentication)

**⚠️ Note**: The following examples require valid credentials.
Without authentication, you'll see error messages (which is expected).

In [ ]:
import time

# Example organization data
org_data = {
    "name": f"demo_org_{int(time.time())}",  # Unique name
    "title": "Demo Organization",
    "description": "A demonstration organization"
}

print("🏢 Creating Organization (requires auth)...")
print(f"📝 Data: {org_data}")

try:
    result = client.register_organization(org_data)
    print(f"\n✅ Organization created: {result}")
    created_org = org_data["name"]
except Exception as e:
    print(f"\n❌ Expected error (no auth): {e}")
    created_org = "demo_org"  # Fallback for examples

In [ ]:
# Example URL resource
url_resource = {
    "resource_name": f"climate_data_{int(time.time())}",
    "resource_title": "Climate Data CSV",
    "owner_org": created_org,
    "resource_url": "https://example.com/climate.csv",
    "file_type": "CSV",
    "notes": "Monthly climate data"
}

print("🌐 Registering URL Resource (requires auth)...")
print(f"📝 Data: {url_resource}")

try:
    result = client.register_url(url_resource)
    print(f"\n✅ URL resource created: {result}")
except Exception as e:
    print(f"\n❌ Expected error (no auth): {e}")

In [ ]:
# Example S3 resource
s3_resource = {
    "resource_name": f"s3_dataset_{int(time.time())}",
    "resource_title": "S3 Dataset",
    "owner_org": created_org,
    "resource_s3": "s3://my-bucket/dataset.parquet",
    "notes": "Large dataset in S3"
}

print("☁️ Registering S3 Resource (requires auth)...")
print(f"📝 Data: {s3_resource}")

try:
    result = client.register_s3_link(s3_resource)
    print(f"\n✅ S3 resource created: {result}")
except Exception as e:
    print(f"\n❌ Expected error (no auth): {e}")

## 7. Error Handling Best Practices

In [ ]:
def safe_api_call(func, *args, description="API call", **kwargs):
    """
    Safely execute an API call with comprehensive error handling.
    """
    try:
        print(f"🔄 Attempting: {description}...")
        result = func(*args, **kwargs)
        print(f"✅ Success: {description}")
        return result
    except ValueError as e:
        print(f"❌ API Error in {description}: {e}")
        return None
    except ConnectionError as e:
        print(f"🌐 Connection Error in {description}: {e}")
        return None
    except Exception as e:
        print(f"⚠️ Unexpected error in {description}: {e}")
        return None

# Examples of safe API calls
print("🛡️ Demonstrating Safe API Calls...")

# Safe organization listing
orgs = safe_api_call(
    client.list_organizations,
    server="global",
    description="listing organizations"
)

# Safe search
results = safe_api_call(
    client.search_datasets,
    ["test"],
    server="global",
    description="searching datasets"
)

print("\n✅ Safe API call demonstrations completed!")

## 8. Complete Workflow Example

In [ ]:
def demo_workflow():
    """Demonstrate a complete data management workflow."""
    
    print("🚀 Complete NDP EP Workflow Demo")
    print("=" * 40)
    
    # Step 1: Explore existing data
    print("\n1️⃣ Exploring existing datasets...")
    try:
        results = client.search_datasets(["climate"], server="global")
        print(f"   📊 Found {len(results)} climate datasets")
    except Exception as e:
        print(f"   ❌ Search failed: {e}")
    
    # Step 2: Check organizations
    print("\n2️⃣ Checking organizations...")
    try:
        orgs = client.list_organizations(server="global")
        print(f"   🏢 Found {len(orgs)} organizations")
        if orgs:
            print(f"   📝 Examples: {', '.join(orgs[:3])}")
    except Exception as e:
        print(f"   ❌ Failed: {e}")
    
    # Step 3: Advanced search
    print("\n3️⃣ Advanced search capabilities...")
    search_examples = [
        {"terms": ["sensor"], "desc": "IoT sensor data"},
        {"terms": ["weather"], "desc": "Weather data"},
    ]
    
    for search in search_examples:
        try:
            results = client.search_datasets(search["terms"], server="global")
            print(f"   🔍 {search['desc']}: {len(results)} results")
        except Exception as e:
            print(f"   ❌ {search['desc']}: {e}")
    
    # Step 4: System health
    print("\n4️⃣ System health check...")
    try:
        status = client.get_system_status()
        print("   ✅ System is operational")
    except:
        print("   ⚠️ System check requires authentication")
    
    print("\n✅ Workflow completed!")
    print("\n💡 With authentication, you could also:")
    print("   - Create organizations")
    print("   - Register datasets (URL, S3, Kafka)")
    print("   - Update and delete resources")
    print("   - Register services")

# Run the demo workflow
demo_workflow()

## 🎯 What's Next?

### To use this library in production:

1. **🔐 Set up authentication** at https://nationaldataplatform.org/
2. **🏢 Create your organization** and manage your data
3. **📊 Register your datasets** using various resource types
4. **🔍 Explore search features** to discover relevant data
5. **📖 Read the documentation** for complete API reference

### Useful Links:

- 📚 [Complete Documentation](https://ndp-ep.readthedocs.io/)
- 💻 [GitHub Repository](https://github.com/sci-ndp/ndp-ep-py)
- 🌐 [NDP Platform](https://nationaldataplatform.org/)
- 📦 [PyPI Package](https://pypi.org/project/ndp-ep/)

### Authentication Example:

```python
# Production setup with environment variables
import os
from ndp_ep import APIClient

client = APIClient(
    base_url=os.getenv("NDP_API_URL", "http://155.101.6.191:8003"),
    token=os.getenv("NDP_API_TOKEN")
)
```

---

**🎉 Thank you for trying the NDP EP Python client!**

*This notebook works interactively in Google Colab and Binder.*